# Day 11: Seating System

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

```text
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

    If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
    If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
    Otherwise, the seat's state does not change.

Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

```text
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

After a second round, the seats with four or more occupied adjacent seats become empty again:

```text
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```

This process continues for three more rounds:

```text
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
```

```text
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
```

```text
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [1]:
# Python imports
from pathlib import Path
from typing import Callable, List

import numpy as np

We approach this by loading the seating plan from a file into a `numpy` array. We represent floor `.` as 0, an empty seat `L` as 1, and an occupied seat `#` as 2.

There are two functions: `parse_seat_line()` processes a line from the file into a list of `int`s, and `load_seat_plan()` turns the file into a `numpy` array.

In [2]:
def parse_seat_line(line: str) -> List[int]:
    """Return a list of ints representing a row in the seating plan
    
    :param line:  a line from the seating plan
    
    Maps . to 0, L to 1, # to 2
    """
    seat_line = []
    for char in line:
        if char == ".":  # floor
            seat_line.append(0)
        if char == "L":  # empty seat
            seat_line.append(1)
        if char == "#":  # occupied
            seat_line.append(2)
    return seat_line

In [3]:
def load_seat_plan(fpath: str) -> np.array:
    """Returns numpy array representing seating plan in passed file
    
    :param fpath:  path to file describing seating plan
    
    Maps . to 0, L to 1, # to 2
    """
    seats = []
    with Path(fpath).open("r") as ifh:
        for line in ifh.readlines():
            seats.append(parse_seat_line(line))
    return np.array(seats)

In [4]:
def update_seats(seats: np.array) -> np.array:
    """Return seat plan array, updated according to part one rules
    
    :param seats: the seating plan as a numpy array
    
    Checks the immediate neighbourhood of each seat in the plan
    (8 seats), and:
    
    - makes an empty seat occupied if there are no occupied seats in
      the neighbourhood
    - empties an occupied seat if there are at least four occupied
      seats in the neighbourhood
    """
    next_seats = seats.copy()  # updated seating plan
    rows, cols = seats.shape  # need limits on array
    
    # iterate over each element in the array
    for ridx in np.arange(rows):
        for cidx in np.arange(cols):
            
            # get immediate neighbourhood
            nbrs = seats[max(0, ridx-1): min(rows, ridx+2), max(0, cidx-1): min(cols, cidx+2)]
            
            # count empty and occupied seats
            empty = np.sum(nbrs == 1) - (1 if seats[ridx, cidx] == 1 else 0)
            occupied = np.sum(nbrs == 2) - (1 if seats[ridx, cidx] == 2 else 0)
            
            # apply rules to get updated array
            # neighbourhood empty, current seat empty -> fill seat
            if occupied == 0 and seats[ridx, cidx] == 1:
                next_seats[ridx, cidx] = 2
            # 4+ neighbouring occupants, current seat full -> empty seat
            elif occupied >= 4 and seats[ridx, cidx] == 2:
                next_seats[ridx, cidx] = 1
                
    return next_seats

We first test the update rule on the example data set

In [5]:
seats = load_seat_plan("day11_test.txt")
seats = update_seats(seats)
seats = update_seats(seats)
seats

array([[2, 0, 1, 1, 0, 1, 2, 0, 2, 2],
       [2, 1, 1, 1, 1, 1, 1, 0, 1, 2],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [2, 1, 1, 1, 0, 1, 1, 0, 1, 2],
       [2, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [2, 0, 1, 1, 1, 1, 2, 0, 2, 2],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [2, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 0, 1, 1, 1, 1, 1, 1, 0, 1],
       [2, 0, 2, 1, 1, 1, 1, 0, 2, 2]])

This produces the correctly updated plan according to the problem above. Now we need a function to update a given plan until no further changes are seen (it reaches equilibrium).

In [6]:
def find_seat_equilibrium(fpath: str, update_fn: Callable) -> int:
    """Return the number of occupied seats, when updating reaches equilibrium
    
    :param fpath:  path to file with seating plan
    :param update_fn:  function updating the seating plan
    """
    # initialise seat plan, and first update
    seats = load_seat_plan(fpath)
    new_seats = update_fn(seats)

    # keep updating while the update and the last-seen seating plan differ
    while True:
        seats = new_seats.copy()
        new_seats = update_fn(seats)
        if np.array_equal(seats, new_seats):
            break

    return np.sum(new_seats == 2)

Solving the test data.

In [7]:
find_seat_equilibrium("day11_test.txt", update_seats)

37

Solving the real puzzle.

In [8]:
find_seat_equilibrium("day11_data.txt", update_seats)

2164

## Part Two

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

```text
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

```text
.............
.L.L.#.#.#.#.
.............
```

The empty seat below would see no occupied seats:

```text
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

```text
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

```text
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

```text
#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
```

```text
#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
```

```text
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
```

```text
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

```text
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

The new update rule is a little more complicated. For each of the eight directions we need to check, we define a function that returns `True` if the first seat is occupied, and `False` if there are no seats, or the first seat is unoccupied.

In [9]:
def check_u(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'up' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    for r in np.arange(ridx-1, -1, -1):
        if array[r, cidx] == 1:
            return False
        if array[r, cidx] == 2:
            return True
    return False

    
def check_d(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'down' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    rows, cols = array.shape
    for r in np.arange(min(ridx + 1, rows), rows):
        if array[r, cidx] == 1:
            return False
        if array[r, cidx] == 2:
            return True
    return False

    
def check_l(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'left' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    for c in np.arange(cidx-1, -1, -1):
        if array[ridx, c] == 1:
            return False
        if array[ridx, c] == 2:
            return True
    return False
    
def check_r(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'right' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    rows, cols = array.shape
    for c in np.arange(min(cidx + 1, cols), cols):
        if array[ridx, c] == 1:
            return False
        if array[ridx, c] == 2:
            return True
    return False

def check_dr(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'down right' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    rows, cols = array.shape
    ridx += 1
    cidx += 1
    while ridx < rows and cidx < cols:
        if array[ridx, cidx] == 1:
            return False
        if array[ridx, cidx] == 2:
            return True
        ridx += 1
        cidx += 1        
    return False

def check_dl(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'down left' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    rows, cols = array.shape
    ridx += 1
    cidx -= 1
    while ridx < rows and cidx > -1:
        if array[ridx, cidx] == 1:
            return False
        if array[ridx, cidx] == 2:
            return True
        ridx += 1
        cidx -= 1
    return False

def check_ur(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'up right' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    rows, cols = array.shape
    ridx -= 1
    cidx += 1
    while ridx > -1 and cidx < cols:
        if array[ridx, cidx] == 1:
            return False
        if array[ridx, cidx] == 2:
            return True
        ridx -= 1
        cidx += 1
    return False
    
def check_ul(array: np.array, ridx: int, cidx: int) -> bool:
    """Return True if first 'up left' seat is occupied, False otherwise
    
    :param array:  seat plan as numpy array
    :param ridx:  row index of seat to check
    :param cidx: col index of seat to check
    """
    rows, cols = array.shape
    ridx -= 1
    cidx -= 1
    while ridx > -1 and cidx > -1:
        if array[ridx, cidx] == 1:
            return False
        if array[ridx, cidx] == 2:
            return True
        ridx -= 1
        cidx -= 1
    return False

def update_seats(seats: np.array) -> np.array:
    """Update seating array with rules from part 2
    
    :param seats:  seating plan as numpy array
    """
    next_seats = seats.copy()
    rows, cols = seats.shape
    
    # Check each seat in the plan
    for ridx in np.arange(rows):
        for cidx in np.arange(cols):
            occupied = 0  # visible occupied seats

            # Check in each direction and update the occupied count
            if check_l(seats, ridx, cidx):
                occupied += 1

            if check_r(seats, ridx, cidx):
                occupied += 1

            if check_u(seats, ridx, cidx):
                occupied += 1
        
            if check_d(seats, ridx, cidx):
                occupied += 1
        
            if check_ul(seats, ridx, cidx):
                occupied += 1

            if check_ur(seats, ridx, cidx):
                occupied += 1
            
            if check_dl(seats, ridx, cidx):
                occupied += 1
            
            if check_dr(seats, ridx, cidx):
                occupied += 1
                
            # Apply rules for part two
            if occupied == 0 and seats[ridx, cidx] == 1:
                next_seats[ridx, cidx] = 2
            elif occupied >= 5 and seats[ridx, cidx] == 2:
                next_seats[ridx, cidx] = 1
                
    return next_seats

We check the first two updates against the puzzle definition.

In [10]:
seats = load_seat_plan("day11_test.txt")
seats = update_seats(seats)
seats = update_seats(seats)
seats

array([[2, 0, 1, 1, 0, 1, 1, 0, 1, 2],
       [2, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 0, 1, 1, 1, 1, 1, 0, 1, 1],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
       [2, 0, 1, 1, 1, 1, 1, 1, 0, 1],
       [2, 0, 1, 1, 1, 1, 1, 0, 1, 2]])

This seems to work, so we solve the test puzzle.

In [11]:
find_seat_equilibrium("day11_test.txt", update_seats)

26

Then we solve the real puzzle.

In [12]:
find_seat_equilibrium("day11_data.txt", update_seats)

1974